In [1]:
# KBS 뉴스 Crawling

from bs4 import BeautifulSoup as bs
# from selenium import webdriver as wd
# from selenium.webdriver.chrome.service impaort Service
# from webdriver_manager.chrome import ChromeDriverManager
import requests
# from urllib.request import urlopen
# import subprocess
import numpy as np
import pandas as pd

from time import sleep

In [2]:
# path_driver = "C:/Users/User/Code/-Diving-into-the-news/Data_Crawling/Driver/chromedriver" # chrome driver path
# subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"')
# options = wd.ChromeOptions()
# # options.add_experimental_option("excludeSwitches", ["enable-logging"])
# options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
# driver = wd.Chrome(path_driver, chrome_options=options)
# driver.implicitly_wait(10)
# # service=Service(ChromeDriverManager().install())

# headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.39.132 Safari/53.36'}

# url = "https://news.sbs.co.kr"

# url_start = "https://news.sbs.co.kr/news/newsflash.do?pageDate=20170101"
# driver.get(url_start)

c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  
c:\ProgramData\Anaconda3\envs\DINS\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [2]:
url = "https://news.sbs.co.kr"
url_main = "https://news.sbs.co.kr/news/newsflash.do?pageDate=20160101"
url_target = "https://news.sbs.co.kr/news/newsflash.do?pageDate=20160101"

In [3]:
columns = ['title', 'headline', 'date', 'news_link', 'content', 'category']
data = []

In [ ]:
print('----- Start -----')

n_page = 0

while True:
    current_page = requests.get(url=url_main) # main page(idx=1)
    while True:
        if current_page.status_code == 200:
            current_page = current_page.text
            break
        else:
            print(current_page.status_code)
        
    soup = bs(current_page, 'html.parser')
    sleep(0.5)
    
    link = soup.find('link', {'rel' : 'canonical'})['href']
    date = link.split('?pageDate=')[-1]
    if date == '20220920':
        break
    
    next_date = list(soup.find('ol',  {'class' : 'mdp_date_list'})) # ['\n', date]
    next_date = [e for e in next_date if e != '\n'][1] # remove '\n'
    next_url = next_date.find('a')['href']
    
    idx_p = 1
    
    n_page += 1
    print()
    print('page: ' + str(n_page))
    print()
    
    while True:
        print()
        print('pageIdx=' + str(idx_p))
        print()
        
        current_page = requests.get(url=url_target) # next page(idx > 1)
        while True:
            if current_page.status_code == 200:
                current_page = current_page.text
                break
            else:
                print(current_page.status_code)
            
        soup = bs(current_page, 'html.parser')
        sleep(5)
        
        flag = soup.find('div', {'class' : 'w_news_list type_issue'}).text.strip() # content presence
        if flag == '해당 일자의 뉴스 정보가 없습니다.':
            break
        
        list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'})) # current page news
        
        for news in list_news:
            try:
                title = news.find('strong').text # title
                headline = news.find('span', {'class' : 'read'}).text # headline
                # thumb = news.find('img')['src'] # thumbnail
                date = news.find('span', {'class' : 'date'}).text # date
                link_news = news.find('meta')['itemid'] # news link
            except AttributeError as e:
                title = None
                headline = None
                date = None
                link_news = None
            
            article_page = requests.get(url=link_news) # news page
            while True:
                if article_page.status_code == 200:
                    article_page = article_page.text
                    break
                else:
                    print(current_page.status_code)
                
            soup2 = bs(article_page, 'html.parser')
            sleep(5)
            
            try:
                content = soup2.find('div', {'itemprop' : 'articleBody'}).text.replace('\n', '') # content
                category = soup2.find('li', {'class' : 'cate03'}).text # category
            except AttributeError as e:
                content = None
                category = None

            if title != None and headline != None and date != None and link_news != None and content != None and category != None:
                data.append([title, headline, date, link_news, content, category])
            
            print(len(data))
        
        idx_p += 1
        page_idx = '&pageIdx=' + str(idx_p)
        url_target = url_main + page_idx
            
    url_main = url + next_url

print()
print('----- End -----')

In [11]:
# print('----- Start -----')

# n_page = 0

# while True:
#     current_page = urlopen(url=url_main) # main page(idx=1)
#     while True:
#         if current_page.code == 200:
#             break
#         else:
#             print(current_page.code)
        
#     soup = bs(current_page, 'html.parser')
#     sleep(0.5)
    
#     link = soup.find('link', {'rel' : 'canonical'})['href']
#     date = link.split('?pageDate=')[-1]
#     if date == '20220920':
#         break
    
#     next_date = list(soup.find('ol',  {'class' : 'mdp_date_list'})) # ['\n', date]
#     next_date = [e for e in next_date if e != '\n'][1] # remove '\n'
#     next_url = next_date.find('a')['href']
    
#     idx_p = 1
    
#     n_page += 1
#     print()
#     print('page: ' + str(n_page))
#     print()
    
#     while True:
#         print()
#         print('pageIdx=' + str(idx_p))
#         print()
        
#         current_page = urlopen(url=url_target) # next page(idx > 1)
#         while True:
#             if current_page.code == 200:
#                 break
#             else:
#                 print(current_page.code)
            
#         soup = bs(current_page, 'html.parser')
#         sleep(5)
        
#         flag = soup.find('div', {'class' : 'w_news_list type_issue'}).text.strip() # content presence
#         if flag == '해당 일자의 뉴스 정보가 없습니다.':
#             break
        
#         list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'})) # current page news
        
#         for news in list_news:
#             try:
#                 title = news.find('strong').text # title
#                 headline = news.find('span', {'class' : 'read'}).text # headline
#                 # thumb = news.find('img')['src'] # thumbnail
#                 date = news.find('span', {'class' : 'date'}).text # date
#                 link_news = news.find('meta')['itemid'] # news link
#             except AttributeError as e:
#                 title = None
#                 headline = None
#                 date = None
#                 link_news = None
            
#             article_page = urlopen(url=link_news) # news page
#             while True:
#                 if article_page.code == 200:
#                     break
#                 else:
#                     print(current_page.code)
                
#             soup2 = bs(article_page, 'html.parser')
#             sleep(10)
            
#             try:
#                 content = soup2.find('div', {'itemprop' : 'articleBody'}).text.replace('\n', '') # content
#                 category = soup2.find('li', {'class' : 'cate03'}).text # category
#             except AttributeError as e:
#                 content = None
#                 category = None

#             if title != None and headline != None and date != None and link_news != None and content != None and category != None:
#                 data.append([title, headline, date, link_news, content, category])
            
#             print(len(data))
        
#         idx_p += 1
#         page_idx = '&pageIdx=' + str(idx_p)
#         url_target = url_main + page_idx
            
#     url_main = url + next_url

# print()
# print('----- End -----')

----- Start -----

page: 1


pageIdx=1

1
2
3
4
5
6
7
8
9
10

pageIdx=2

11
12
13
14
15
16
17
18
19
20

pageIdx=3

21
22
23
24
25
26
27
28
29
30

pageIdx=4

31
32
33
34
35
36
37
38
39
40

pageIdx=5

41
42
43


In [8]:
# res = requests.get(url=url_start).text
# soup = bs(res, 'html.parser')
# link = soup.find('link', {'rel' : 'canonical'})['href']
# date = link.split('?pageDate=')[-1]
# date

'20170101'

In [4]:
# print('----- Start -----')

# n_page = 0

# while True:
#     date = driver.current_url.split('pageDate=')[-1]
#     if date == '20220920':
#         break
    
#     current_page = driver.page_source
#     soup = bs(current_page, 'html.parser')
    
#     next_date = list(soup.find('ol',  {'class' : 'mdp_date_list'})) # ['\n', date]
#     next_date = [e for e in next_date if e != '\n'][1] # remove '\n'
#     next_url = next_date.find('a')['href']
    
#     idx_p = 2
#     url_start = driver.current_url
    
#     n_page += 1
#     print()
#     print(n_page)
#     print()
    
#     while True:
#         flag = soup.find('div', {'class' : 'w_news_list type_issue'}).text.strip() # content presence
#         if flag == '해당 일자의 뉴스 정보가 없습니다.':
#             break
        
#         list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'})) # current page news
        
#         for news in list_news:
#             title = news.find('strong').text # title
#             headline = news.find('span', {'class' : 'read'}).text # headline
#             # thumb = news.find('img')['src'] # thumbnail
#             date = news.find('span', {'class' : 'date'}).text # date
#             link_news = news.find('meta')['itemid'] # news link
            
#             article_page = requests.get(url=link_news).text # news page
#             sleep(10)
#             soup2 = bs(article_page, 'html.parser')
            
#             content = soup2.find('div', {'itemprop' : 'articleBody'}).text.replace('\n', '') # content
#             category = soup2.find('li', {'class' : 'cate03'}).text # category
            
#             data.append([title, headline, date, link_news, content, category])
            
#             print(len(data))
        
#         page_idx = '&pageIdx=' + str(idx_p)
#         driver.get(url_start + page_idx)
#         idx_p += 1
            
#     url_next = url + next_url
#     driver.get(url_next)
#     sleep(10)

# driver.quit()
# print()
# print('----- End -----')

----- Start -----

1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


In [ ]:
dataset = pd.DataFrame(data=data, columns=columns)
dataset.to_csv('./kbs.csv', index=False)

In [ ]:
# span class read : 헤드라인 ~
# span class thumb : 썸네일 ~
# strong class sub : 뉴스 제목 ~
# span class data : 뉴스 날짜 및 시간 ~ 
# meta itemid : 뉴스 링크 ~


# li class cate03 : 카테고리 ~
# div class itemprop articleBody : 뉴스 내용

# div class mdp_inner -> a href #page : 
# div class w_news_list type_issue : 뉴스 정보

In [10]:
current_page = urlopen(url = url_main)
soup = bs(current_page, 'html.parser')
soup

# list_news = list(soup.find_all('li', {'itemtype' : 'http://schema.org/NewsArticle'}))

# link_news = list_news[0].find('img')['src'] 



<!DOCTYPE html>

<html itemscope="" itemtype="http://schema.org/WebPage" lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="SBS" name="attribution"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>SBS 뉴스 최신/속보</title>
<meta content="SBS 뉴스 최신/속보" name="title"/>
<meta content="width=970" name="viewport"/>
<link href="https://img.sbs.co.kr/news/logo/sbslogo.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://news.sbs.co.kr/news/newsFlash.do?pageDate=20170101" rel="canonical"/>
<link href="http://news.sbs.co.kr/news/newsFlash.do?pageDate=20170101" rel="alternate"/>
<link href="https://mnews.sbs.co.kr/news/newsFlash.do" media="only screen and (max-width: 640px)" rel="alternate"/>
<link crossorigin="" href="https://image.sbs.co.kr" rel="preconnect"/><link crossorigin="" href="https://img.sbs.co.kr" rel="preconnect"/>
<link href="https://image.sbs.co.kr/" rel="dns-prefetch"/><link href="https://img.sbs.co.kr/" rel="dns-prefetch"/><link href="https://api.da